## Part A: Data Collection 

RB Player Data is getting web scraped from ProFootballReference site between the years 2014-2024


### Websites Used For Support:<br>
- [BrowserStack - Download File using Selenium](https://www.browserstack.com/guide/download-file-using-selenium-python)
- [GeeksForGeeks - Scrape and Save Table using Selenium](https://www.geeksforgeeks.org/scrape-and-save-table-data-in-csv-file-using-selenium-in-python/#)
- [RealPython - Modern Web Automation with Selenium](https://realpython.com/modern-web-automation-with-python-and-selenium/#locate-elements-in-the-dom) 
- [StackOverflow - Wait for file to be downloaded in Selenium](https://stackoverflow.com/questions/63637077/how-to-wait-for-a-file-to-be-downloaded-in-selenium-and-python-before-moving-for)

Semi-Automated Data Extraction:

Selenium opens Google Chrome,<br>
User downloads the Excel file,<br>
File renamed according to offset in its URL

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import os, time

#File download Paths
selenium_profile_path = r"C:\SeleniumProfiles\StatheadSession"
download_dir = os.path.join(os.getcwd(), "selenium_downloads")
os.makedirs(download_dir, exist_ok=True)

# Chrome Options Setup
options = uc.ChromeOptions()
options.user_data_dir = selenium_profile_path
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
prefs = {"download.default_directory": download_dir,
         "download.prompt_for_download": False,
         "directory_upgrade": True,
         "safebrowsing.enabled": True}
options.add_experimental_option("prefs", prefs)

driver = uc.Chrome(options=options, user_data_dir=selenium_profile_path, headless=False)

# Detect if download is finished
def download_complete():
    print("Checking if download is complete")
    return not any(f.endswith(".crdownload") for f in os.listdir(download_dir))


max_rows = 71000 #Estimated finished based on website query
base_url = f"https://stathead.com/football/player-game-finder.cgi?request=1&timeframe=seasons&match=player_game&qb_start_num_career_max=400&season_end=-1&rookie=N&team_game_num_season_min=1&weight_max=500&comp_type=reg&qb_start_num_career_min=1&player_game_num_career_min=1&draft_pick_type=overall&player_game_num_career_max=400&year_min=2014&year_max=2024&season_start=1&season_positions[]=rb&player_game_num_season_min=1&week_num_season_max=22&team_game_num_season_max=17&week_num_season_min=1&player_game_num_season_max=18&order_by=fantasy_points&cstat[1]=rush_att&ccomp[1]=gt&cval[1]=1"
print(f"File will populate here: {download_dir}")
files_preDownload = set(os.listdir(download_dir))
print(f"Content before download: {files_preDownload}")
for offset in range (0, max_rows, 200):
    try:
        url = base_url + f"&offset={offset}"
        driver.get(url)
        print(f"🟢 Opened URL: {url}")
        time.sleep(10)

        #Time Delay to allow user to click 'download'
        while not download_complete():
            time.sleep(5)

        # Rename file
        files_postDownload = set(os.listdir(download_dir))
        print("Prepping for post download workflow")
        new_file = (files_postDownload - files_preDownload)
        new_xlsx_files = {f for f in new_file if f.endswith(".xls") or f.endswith(".xlsx")}
        if len(new_xlsx_files) == 1:
            original_name = new_xlsx_files.pop()
            new_name = f"Weekly-NFL-RB_stats({offset}).xlsx"
            os.rename(
                os.path.join(download_dir, original_name),
                os.path.join(download_dir, new_name)
            )
            print(f"Renamed: {original_name} → {new_name}")
        elif len(new_xlsx_files) > 1:
            print(f"Multiple new files detected: {new_xlsx_files}. Skipping rename")
        else:
            print("No new file detected")
        files_preDownload = set(os.listdir(download_dir))
    except Exception as e:
        print(f"An error occured: {e}")

driver.quit()
print("Complete, closing Chrome")


html_to_df(): Converting all 'Excel' files that turned out to be in HTML format into a single combined pandas dataframe for simpler data manipulation

Print statements to evaluate the data better

In [15]:
import os
import pandas as pd

def html_to_df(path):
    weekly_rb_stats = []
    htmlFiles = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(").xlsx")]
    for file in htmlFiles:
        try:
            df = pd.read_html(file,header=[0,1])[0]
            weekly_rb_stats.append(df)
        except ValueError:
            print(f"No valid tables found in {file}")
        except Exception as e:
            print(f"Error processing file: {e}")
    weekly_rb_stats = pd.concat(weekly_rb_stats)
    return weekly_rb_stats
    

download_dir = os.path.join(os.getcwd(), "selenium_downloads")
weekly_stats_df = html_to_df(download_dir)
print(f"Preview of Weekly RB Stats: {weekly_stats_df.head()}")
print(f"\n Statistical Summary of Weekly RB Stats: {weekly_stats_df.describe()}")
print(f"\n Size of Weekly RB Stats Dataframe: {weekly_stats_df.shape}")
weekly_stats_df.info()

Preview of Weekly RB Stats:   Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player             FantPt                Att   
0                  1       Alvin Kamara               53.2                 22   
1                  2          Joe Mixon               51.1                 22   
2                  3    Jonathan Taylor               50.4                 32   
3                  4       Le'Veon Bell               47.8                 38   
4                  5      Derrick Henry               47.8                 17   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0 Unnamed: 7_level_0  \
                 Day                 G#               Week               Date   
0                Fri                 15                 16         2020-12-25   
1                Sun                  9                  9         2022-11-06   
2                Sun                 11                 11         2021-11-21   

### Stadium Mapping from 2014-2024 <br>

Note: Houston Texans switched from natural grass to artificial turf shortly before Week 2 of 2015 season.  For all intensive purposes, will assume all 2015 season was played on turf

Source(s):<br><br>
 - [ESPN NFL Stadium Surface Types](https://www.espn.com/nfl/story/_/id/38565107/nfl-stadium-surfaces-strategies-challenges-faqs) <br>
 - [Sports Illustrated Stadium Surface Types](https://www.si.com/nfl/2015/09/29/nfl-stadium-turf-grass-rankings#:~:text=Let's%20revisit%20the%20formula%20that,recovery%20plays%20well%20in%20Charlotte.)<br>
 - [Baltimore Ravens switch to grass](https://www.baltimoreravens.com/news/ravens-switching-to-natural-grass-at-m-t-bank-stadium-16430494) <br>
- [Houston Texans switch to grass](https://www.houstontexans.com/news/texans-to-play-on-artificial-turf-for-rest-of-2015-15899086#:~:text=For%20the%20remainder%20of%20the,September%2027%20against%20Tampa%20Bay.) <br>
- [OAK stadium](https://turfgrasssod.org/raiders-maintain-the-tradition-of-football-with-natural-grass-field/)<br>
- [TEN switch to turf](https://www.tennesseetitans.com/news/why-the-titans-are-switching-to-turf-at-nissan-stadium-starting-in-2023)<br>
- [Wembley(London) Stadium Surface Type](https://www.profootballnetwork.com/is-wembley-stadium-turf-or-grass/)<br>
- [Twickenham Stadium Surface Type](https://www.nflweather.com/stadium/twikenham-stadium#)<br>
- [Estadio Stadium Surface Type](https://www.nflweather.com/stadium/estadio-azteca)<br>
- [Tottenham Stadium Surface Type](https://www.nflweather.com/stadium/tottenham-hotspur-stadium) <br>
- [Allianz Arena Surface Type](https://www.nflweather.com/stadium/allianz-arena) <br>
- [Frankfurt Stadium Surface Type](https://www.nflweather.com/stadium/frankfurt-stadium) <br>
- [Corinthians Arena Surface Type](https://www.nflweather.com/stadium/corinthians-arena) <br>

In [ ]:
stadium_surface_dict = {
    'ARI': 'grass',
    'ATL': 'turf', 
    'BAL': 'grass',  #add to exceptions - had artificial turf in 2016 (X)
    'BUF': 'grass',
    'CAR': 'grass', #add to exceptions - had turf from 2021 onwards (X)
    'CHI': 'grass',
    'CIN': 'turf',
    'CLE': 'grass',
    'DAL': 'turf',
    'DEN': 'grass',
    'DET': 'turf',
    'GNB': 'grass',
    'HOU': 'turf', #add to exceptions - had grass in 2014/2015 (X)
    'IND': 'turf',
    'JAX': 'grass', 
    'KAN': 'grass', 
    'LAC': 'turf', 
    'LAR': 'turf', 
    'LVR': 'grass', 
    'MIA': 'grass',
    'MIN': 'turf', 
    'NOR': 'turf',
    'NWE': 'turf',
    'NYG': 'turf', 
    'NYJ': 'turf',
    'OAK': 'grass',
    'PHI': 'grass',
    'PIT': 'grass',
    'SDG': 'grass',
    'SEA': 'turf',
    'SFO': 'grass', 
    'STL': 'turf',
    'TAM': 'grass',
    'TEN': 'grass', #add to exceptions - switched to turf in 2023 (X)
    'WAS': 'grass'
}

#Defining exceptions where teams had a different field type for a few seasons before transition
stadium_surface_exceptions_dict = {
    (2023, 'TEN'): 'turf',
    (2024, 'TEN'): 'turf', 
    (2014, 'HOU'): 'grass',
    (2021, 'CAR'): 'turf',
    (2022, 'CAR'): 'turf',
    (2023, 'CAR'): 'turf',
    (2024, 'CAR'): 'turf',
    (2014, 'BAL'): 'turf',
    (2015, 'BAL'): 'turf'
}

#Dictionary will list designated 'home' team as the last value for surface type key
#Setup will be (year, season week, 'home team'): 'field type'
int_games_dict = {
    (2014, 4, 'OAK'): 'turf',
    (2014, 8, 'ATL'): 'turf',
    (2014, 10, 'JAX'): 'turf',
    (2015, 4, 'MIA'): 'turf',
    (2015, 7, 'JAX'): 'turf',
    (2015, 8, 'KAN'): 'turf',
    (2016, 4, 'JAX'): 'turf',
    (2016, 7, 'LAR'): 'grass', #Twickenham stadium in London
    (2016, 8, 'CIN'): 'turf',
    (2016, 10, 'OAK'): 'grass', #Mexico stadium
    (2017, 3, 'JAX'): 'turf',
    (2017, 4, 'MIA'): 'turf',
    (2017, 7, 'LAR'): 'grass', #Twickenham stadium 
    (2017, 8, 'CLE'): 'grass', #Twickenham stadium 
    (2017, 10, 'OAK'): 'grass', #Mexico stadium
    (2018, 6, 'OAK'): 'turf',
    (2018, 7, 'LAC'): 'turf',
    (2018, 8, 'JAX'): 'turf',
    (2019, 5, 'OAK'): 'grass', #Tottenham stadium
    (2019, 6, 'TAM'): 'grass', #Tottenham stadium
    (2019, 8, 'LAR'): 'turf',
    (2019, 9, 'JAX'): 'turf',
    (2019, 11, 'LAC'): 'grass', #Mexico stadium
    (2021, 5, 'ATL'): 'grass', #Tottenham stadium
    (2021, 6, 'JAX'): 'grass', #Tottenham stadium
    (2022, 4, 'NOR'): 'grass', #Tottenham stadium
    (2022, 5, 'GNB'): 'grass', #Tottenham stadium
    (2022, 8, 'JAX'): 'turf',
    (2022, 10, 'TAM'): 'grass', #Allianz Arena
    (2022, 11, 'ARI'): 'grass', #Mexico stadium
    (2023, 4, 'JAX'): 'turf',
    (2023, 5, 'BUF'): 'grass', #Tottenham stadium
    (2023, 6, 'TEN'): 'grass', #Tottenham stadium
    (2023, 9, 'KAN'): 'grass', #Frankfurt stadium
    (2023, 10, 'NWE'): 'grass', #Frankfurt stadium
    (2024, 1, 'PHI'): 'grass', #Brazil stadium
    (2024, 5, 'MIN'): 'grass', #Tottenham stadium
    (2024, 6, 'CHI'): 'grass', #Tottenham stadium
    (2024, 7, 'JAX'): 'grass', #Tottenham stadium
    (2024, 10, 'CAR'): 'grass' #Allianz Arena   
}

def surfaceObtainer(team, year, week=None):
    if week and (year, week, team) in int_games_dict:
        return int_games_dict[(year, week, team)]
    return stadium_surface_exceptions_dict.get((year, team)) or stadium_surface_dict.get(team)